In [1]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [2]:
from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)


download("https://github.com/AllenDowney/ThinkStats/raw/v3/nb/thinkstats.py")

<IPython.core.display.Javascript object>

In [4]:
try:
    import empiricaldist
except ImportError:
    !pip install empiricaldist

<IPython.core.display.Javascript object>

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from thinkstats import decorate

<IPython.core.display.Javascript object>

In [7]:
import json
import pandas as pd

def parse_elec_usage_file(filepath):
    with open(filepath, 'r') as file:
        lines = file.readlines()
        
    # Join the lines into a single string and attempt to fix any broken JSON
    data_str = ''.join(lines)
    
    # Split into series based on the presence of "series_id"
    series_data = data_str.split('{"series_id":')
    
    # Initialize a list to store each series as a dictionary
    series_list = []
    
    for i, series in enumerate(series_data):  # Skip the first element since it's empty
        series_json_str = '{"series_id":' + series.strip()
        
        # Clean up the JSON string by removing trailing commas and fixing brackets
        series_json_str = series_json_str.rstrip(',')  # Remove trailing comma
        if series_json_str[-1] != '}':
            series_json_str += '}'  # Ensure the string ends with a closing bracket
        
        try:
            series_dict = json.loads(series_json_str)
            series_list.append(series_dict)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            continue
            
        if i > 100:
            break

    # Create a DataFrame with relevant metadata fields
    metadata_fields = ["series_id", "name", "units", "f", "copyright", "source", 
                       "iso3166", "lat", "lon", "geography", "start", "end", "last_updated"]

    df_metadata = pd.DataFrame(series_list, columns=metadata_fields)
    
    return df_metadata

# Example usage:
df = parse_elec_usage_file('ELEC.txt')
df.head()


Error decoding JSON: Expecting value: line 1 column 14 (char 13)


,series_id,name,units,f,copyright,source,iso3166,lat,lon,geography,start,end,last_updated
0,ELEC.PLANT.GEN.54276-DFO-ST.M,Net generation : Univ of NC Chapel Hill Cogen ...,megawatthours,M,None,"EIA, U.S. Energy Information Administration",USA-NC,35.9069,-79.0617,USA-NC,200401,202406,2024-08-23T16:44:50-04:00
1,ELEC.PLANT.GEN.54276-DFO-ST.Q,Net generation : Univ of NC Chapel Hill Cogen ...,megawatthours,Q,None,"EIA, U.S. Energy Information Administration",USA-NC,35.9069,-79.0617,USA-NC,2004Q1,2024Q2,2024-08-23T16:44:50-04:00
2,ELEC.PLANT.GEN.54276-NG-ALL.M,Net generation : Univ of NC Chapel Hill Cogen ...,megawatthours,M,None,"EIA, U.S. Energy Information Administration",USA-NC,35.9069,-79.0617,USA-NC,200101,202406,2024-08-23T16:44:50-04:00
3,ELEC.PLANT.GEN.54276-NG-ALL.Q,Net generation : Univ of NC Chapel Hill Cogen ...,megawatthours,Q,None,"EIA, U.S. Energy Information Administration",USA-NC,35.9069,-79.0617,USA-NC,2001Q1,2024Q2,2024-08-23T16:44:50-04:00
4,ELEC.PLANT.GEN.54276-NG-ST.M,Net generation : Univ of NC Chapel Hill Cogen ...,megawatthours,M,None,"EIA, U.S. Energy Information Administration",USA-NC,35.9069,-79.0617,USA-NC,200301,202406,2024-08-23T16:44:50-04:00


<IPython.core.display.Javascript object>

In [8]:
!ls -lh ELEC.txt

-rw-rw-rw- 1 downey downey 1.2G Aug 26 15:39 ELEC.txt


<IPython.core.display.Javascript object>